<a href="https://colab.research.google.com/github/kreyzon/ImersaoAluraIA/blob/main/Gerador_de_MVP_com_Agentes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pip -q install google-genai

In [1]:
# Configura a API Key do Google Gemini

import os
from google.colab import userdata

os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

In [33]:
# Configura o cliente da SDK do Gemini

from google import genai

client = genai.Client()
# para consultar outras opções de modelo
# for model in client.models.list():
#   print(model.name)
MODEL_SIMPLES = "gemini-2.0-flash"
MODEL_COMPLETO = "gemini-2.5-pro-exp-03-25"

In [5]:
# Instalar Framework de agentes do Google ################################################
!pip install -q google-adk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 49.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.1/232.1 kB 16.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 217.1/217.1 kB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 334.1/334.1 kB 22.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.1/125.1 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.0/119.0 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.9/194.9 kB 15.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.

In [6]:
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from google.genai import types  # Para criar conteúdos (Content e Part)
from datetime import date
import textwrap # Para formatar melhor a saída de texto
from IPython.display import display, Markdown # Para exibir texto formatado no Colab
import requests # Para fazer requisições HTTP
import json
import warnings

warnings.filterwarnings("ignore")

In [7]:
# Função auxiliar que envia uma mensagem para um agente via Runner e retorna a resposta final
def call_agent(agent: Agent, message_text: str) -> str:
    # Cria um serviço de sessão em memória
    session_service = InMemorySessionService()
    # Cria uma nova sessão (você pode personalizar os IDs conforme necessário)
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")
    # Cria um Runner para o agente
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    # Cria o conteúdo da mensagem de entrada
    content = types.Content(role="user", parts=[types.Part(text=message_text)])

    final_response = ""
    # Itera assincronamente pelos eventos retornados durante a execução do agente
    for event in runner.run(user_id="user1", session_id="session1", new_message=content):
        if event.is_final_response():
          for part in event.content.parts:
            if part.text is not None:
              final_response += part.text
              final_response += "\n"
    return final_response

In [8]:
# Função auxiliar para exibir texto formatado em Markdown no Colab
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [25]:
##########################################
# --- Agente 1: Stakeholder --- #
##########################################
def agente_cliente(tipo_evento):

    cliente = Agent(
        name="agente_cliente",
        model=MODEL_SIMPLES,
        instruction="""
        Você é um cliente de uma empresa que organiza eventos, sendo assim você cria um evento específico como uma festa de casamento, aniversário ou team building.
        Para criar um evento, você deve indicar de forma aleatoria uma data futura de até um mês a frente, qual o horário e uma quantidade de convidados.
        O tipo de evento será informado a seguir.
        """,
        description="Agente que cria uma demanda para uma equipe de desenvolvimento"
    )

    entrada_do_agente_cliente = f"Problema principal: {tipo_evento}"

    detalhes_evento = call_agent(cliente, entrada_do_agente_cliente)
    return detalhes_evento

In [38]:
### teste do agente cliente
# tipo_evento = "Casamento no Porto"
# detalhes_evento = agente_cliente(tipo_evento)
# print("\n--- 📝 Resultado do cliente ---\n")
# display(to_markdown(detalhes_evento))


--- 📝 Resultado do cliente ---



> Ótimo! Para o casamento no Porto, vamos aos detalhes:
> 
> *   **Data:** 18 de Julho de 2024 (uma quinta-feira)
> *   **Horário:** 16:00
> *   **Número de Convidados:** 150


In [66]:
################################################
# --- Agente 2: Auxiliar Administrativo --- #
################################################
def agente_aux_adm(tipo_evento, detalhes_evento):
    aux_adm = Agent(
        name="agente_aux_adm",
        model=MODEL_SIMPLES,
        instruction="""
        Você é um auxiliar administrativo de uma empresa que organiza diversos tipos de eventos e é responsável por receber os pedidos dos clientes.
        Sua função vai ser consultar na internet através da ferramenta de busca do google (google_search) para descobrir o que é necessário para o evento acontecer.
        Exemplos do que deve ser levado em conta para isso:
        - festas de casamentos precisam de um local com salão grande ou que seja open space, mesas e cadeiras, decoração com flores, um bolo de casamento, aperetivos, bebidas alcoolicas e não-alcoolicas, e um DJ.
        - festas de aniversários para menores de idades precisam de um local fechado, mesas e cadeiras, bolo de aniversário, aperetivos, bebidas não-alcoolicas e um DJ.
        - team building precisam de um local open space, mesas e cadeira, aperetivos, bebidas alcoolicas e não-alcoolicas.

        Devolva a resposta em formato que pode ser utilizado com array do python e a saída deve conter estritamente apenas a estrutura de dados do python e que siga o JSON schema a seguir

        {
          "$schema": "http://json-schema.org/draft-07/schema#",
          "title": "Evento de Casamento",
          "description": "Schema para descrever detalhes de um evento de casamento, incluindo elementos essenciais.",
          "type": "object",
          "properties": {
            "Tipo de Evento": {
              "type": "string",
              "description": "O tipo do evento, por exemplo, 'Casamento'."
            },
            "Localização": {
              "type": "string",
              "description": "A localização do evento (cidade e país)."
            },
            "Data": {
              "type": "string",
              "description": "A data do evento."
            },
            "Horário": {
              "type": "string",
              "description": "O horário do evento."
            },
            "Número de Convidados": {
              "type": "integer",
              "description": "O número estimado de convidados."
              // Podemos adicionar "minimum": 0 se for o caso, mas para este exemplo não é estritamente necessário com base nos dados fornecidos.
            },
            "Elementos Essenciais": {
              "type": "array",
              "description": "Uma lista dos elementos essenciais necessários para o evento.",
              "items": {
                "type": "object",
                "properties": {
                  "Categoria": {
                    "type": "string",
                    "description": "A categoria do elemento essencial (ex: 'Local/Espaço para Casamento', 'Catering')."
                  },
                  "Necessidades": {
                    "type": "array",
                    "description": "Uma lista de necessidades ou opções específicas dentro desta categoria.",
                    "items": {
                      "type": "string",
                      "description": "Uma descrição individual de uma necessidade ou opção."
                    }
                  }
                },
                "required": [
                  "Categoria",
                  "Necessidades"
                ]
              }
            }
          },
          "required": [
            "Tipo de Evento",
            "Localização",
            "Data",
            "Horário",
            "Número de Convidados",
            "Elementos Essenciais"
          ]
        }
        """,
        description="Agente que busca o que precisa ",
        tools=[google_search]
    )

    entrada_do_agente_aux_adm = f"Tipo de evento: {tipo_evento}\nDetalhes do evento: {detalhes_evento}"
    # Executa o agente
    plano_evento = call_agent(aux_adm, entrada_do_agente_aux_adm)
    return plano_evento

In [48]:
### teste o agente auxiliar administrativo
# plano_evento = agente_aux_adm(tipo_evento, detalhes_evento)
# print("\n--- 📝 Resultado do auxiliar administrativo ---\n")
# display(to_markdown(plano_evento))


--- 📝 Resultado do auxiliar administrativo ---



> ```json
> {
>   "Tipo de Evento": "Casamento",
>   "Localização": "Porto, Portugal",
>   "Data": "18 de Julho de 2024",
>   "Horário": "16:00",
>   "Número de Convidados": 150,
>   "Elementos Essenciais": [
>     {
>       "Categoria": "Local/Espaço para Casamento",
>       "Necessidades": [
>         "Salão de festas ou espaço ao ar livre que acomode 150 convidados [4, 6, 7]",
>         "Opções de espaços com jardins [6]",
>         "Espaços com serviços de catering [4]",
>         "Espaços com opções de cerimónia civil [8, 17, 21]"
>       ]
>     },
>     {
>       "Categoria": "Catering",
>       "Necessidades": [
>         "Serviço de catering para 150 pessoas [2, 3]",
>         "Opções de menu tradicional português e cozinha de autor [9]",
>         "Bebidas (alcoólicas e não alcoólicas) [2]",
>         "Aperitivos [2]",
>         "Bolo de casamento [6]"
>       ]
>     },
>     {
>       "Categoria": "Decoração",
>       "Necessidades": [
>         "Decoração floral [20]",
>         "Aluguer de mobiliário (mesas, cadeiras) [11, 12]",
>         "Iluminação [11]",
>         "Decoração personalizada [5, 11]"
>       ]
>     },
>     {
>       "Categoria": "Entretenimento",
>       "Necessidades": [
>         "DJ ou banda [2, 10]",
>         "Espaço para dança [6]"
>       ]
>     },
>     {
>       "Categoria": "Serviços Adicionais",
>       "Necessidades": [
>         "Wedding Planner [8, 10]",
>         "Aluguer de carro [19]",
>         "Fotógrafo/Vídeo [4]",
>         "Lembranças para os convidados [19]"
>       ]
>     }
>   ]
> }
> ```


In [67]:
######################################
# --- Agente 3: Analista de orçamentos --- #
######################################
def agente_analista_budget(tipo_evento, categoria, elemento_essencial):
    analista_budget = Agent(
        name="agente_analista_budget",
        model=MODEL_SIMPLES,
        instruction="""
        Você é um analista de orçamentos especializado em uma categoria específica, como fazer orçamento para alugar locais ou orçamento para catering ou orçamento para contratar uma banda.
        Sua função vai ser consultar na internet através da ferramenta de busca do google (google_search) para atender aos elementos essenciais do tipo de evento que lhe foram informados.
        Busque 5 opções de orçamento para a sua categoria e escolha as 3 opções mais baratas.

        Devolva a resposta em formato que pode ser utilizado com array do python e a saída deve conter estritamente apenas a estrutura de dados do python e que siga o JSON schema a seguir
        {
          "$schema": "http://json-schema.org/draft-07/schema#",
          "title": "Lista de Opções",
          "description": "Schema para uma lista de opções, cada uma com nome, preço e características.",
          "type": "array",
          "items": {
            "type": "object",
            "properties": {
              "Opção": {
                "type": "string",
                "description": "O nome ou título da opção."
              },
              "Preço": {
                "type": "string",
                "description": "Informação sobre o preço da opção."
              },
              "Características": {
                "type": "array",
                "description": "Uma lista de características associadas a esta opção.",
                "items": {
                  "type": "string",
                  "description": "Uma descrição individual de uma característica."
                }
              }
            },
            "required": [
              "Opção",
              "Preço",
              "Características"
            ]
          }
        }
        """,
        description="Agente que atua como analista de orçamentos",
        tools=[google_search]
    )
    entrada_do_agente_analista_budget = f"Tipo de evento: {tipo_evento}\nCategoria Orçamento: {categoria}\nElemento essencial: {elemento_essencial}"
    # Executa o agente
    rascunho = call_agent(analista_budget, entrada_do_agente_analista_budget)
    return rascunho

In [59]:
### teste o agente analista de orçamento
# import json
# plano_object = json.loads(plano_evento.replace("```json", "").replace("```", ""))
# budget = agente_analista_budget(tipo_evento, plano_object.get("Elementos Essenciais")[0].get("Categoria"), plano_object.get("Elementos Essenciais")[0].get("Necessidades"))
# print("\n--- 📝 Resultado do analista de orçamento ---\n")
# display(to_markdown(budget))


--- 📝 Resultado do analista de orçamento ---



> ```python
> [
>   {
>     "Opção": "Quinta LR Gardens",
>     "Preço": "A partir de 74€ por pessoa",
>     "Características": [
>       "Acomoda de 50 a 240 convidados [4]",
>       "Rodeada por beleza natural [4]",
>       "Serviço de catering (Gémeos Catering) [4]"
>     ]
>   },
>   {
>     "Opção": "Quinta de Gatão",
>     "Preço": "A partir de 85€ por pessoa",
>     "Características": [
>       "Acomoda de 50 a 120 convidados [3]",
>       "Espaço com beleza e requinte [3]",
>       "Hospedagem antes e depois do evento [3]"
>     ]
>   },
>   {
>     "Opção": "Casa dos Arcos Boavista",
>     "Preço": "A partir de 100€ por pessoa",
>     "Características": [
>       "Acomoda de 70 a 250 convidados [4]",
>       "Arquitetura imponente [4]",
>       "Serviço de decoração à medida [4]"
>     ]
>   }
> ]
> ```


In [95]:
##########################################
# --- Agente 4: Organizador do evento --- #
##########################################

def agente_organizador(tipo_evento, budgets):
    organizador = Agent(
        name="agente_organizador",
        model=MODEL_SIMPLES,
        instruction="""
        Você é o organizador de um evento.
        Sua função inicial será analisar os orçamentos recebidos (budgets) de diversas categorias e montar 3 planos bastante resumidos para apresentar ao cliente, citando o principalmente o valor de cada um deles.
        """,
        description="Agente organizador.",
    )
    entrada_do_agente_organizador = f"Tipo de evento: {tipo_evento}\Lista de Orçamentos: {budgets}\n"
    # Executa o agente
    evento = call_agent(organizador, entrada_do_agente_organizador)
    return evento

In [92]:
def agente_atendente(tipo_evento, evento, msg):
    atendente = Agent(
        name="agente_atendente",
        model=MODEL_SIMPLES,
        instruction="""
        Você é um atendente de um evento e irá conversar com o cliente sobre o evento informado.
        Você irá receber um plano de evento (evento) e uma resposta do cliente (msg) e sempre deixará uma pergunta para o cliente até que ele confirme que está satisfeito com o evento.
        """,
        description="Agente atendente.",
    )
    entrada_do_agente_atendente = f"Tipo de evento: {tipo_evento}\Evento: {evento}\nResposta do cliente"
    # Executa o agente
    resposta_atendente = call_agent(atendente, entrada_do_agente_atendente)
    return resposta_atendente

In [61]:
### teste o agente analista de orçamento
# plano_object = json.loads(plano_evento.replace("```json", "").replace("```", ""))
# budgets = []
# for elemento_essencial in plano_object.get("Elementos Essenciais"):
#   budget = agente_analista_budget(tipo_evento, elemento_essencial.get("Categoria"), elemento_essencial.get("Necessidades"))
#   budgets.append(budget)
# evento = agente_organizador(tipo_evento, budgets)
# print("\n--- 📝 Resultado do organizador do evento ---\n")
# display(to_markdown(evento))


--- 📝 Resultado do organizador do evento ---



> Olá! Com base nos orçamentos que me forneceu, elaborei três planos distintos para o seu casamento no Porto. Os planos foram feitos para serem mais resumidos e informativos, se desejar mais detalhes sobre algum deles, basta perguntar!
> 
> **Plano Essencial**
> 
> *   **Local:** Agueiro (a partir de 30€ por convidado)
> *   **Catering:** Spacegarden (€19,50 por pessoa)
> *   **Decoração:** Florista Lina (a partir de 20€)
> *   **Entretenimento:** DJ (350€ - 800€)
> *   **Total Estimado:** Este plano foca no essencial, ideal para quem procura uma opção econômica sem abrir mão de um dia especial. O custo é bastante variável dependendo do número de convidados e das opções de DJ, mas podemos estimar algo entre 400€ - 900€, sem contar o preço do local por convidado.
> 
> **Plano Elegância**
> 
> *   **Local:** Restaurante Escondidinho Terrace (a partir de 85€ por convidado)
> *   **Catering:** Dream Catering (a partir de 45€ por pessoa)
> *   **Decoração:** Flores de Joaquim Santos (a partir de 35€)
> *   **Entretenimento:** Banda Musical (690€)
> *   **Serviços Adicionais:** Fotógrafo Casório (a partir de 490€)
> 
> *   **Total Estimado:** Este plano oferece um toque de elegância e qualidade superior. O valor total do plano pode variar bastante também dependendo do número de convidados, mas podemos estimar um valor inicial a partir dos 1700€, sem contar o preço do local por convidado.
> 
> **Plano Luxo**
> 
> *   **Local:** Quinta da Morgadinha (a partir de 85€ por convidado)
> *   **Catering:** MiliEventos Catering (preço variável, personalizável)
> *   **Decoração:** Eventos Privé (a partir de 50€)
> *   **Entretenimento:** Global Dança (desde 150€)
> *   **Serviços Adicionais:**
>     *   Wedding Planner Meraki (desde 400€)
>     *   Aluguer de Carro Clássico (140€)
>     *   Vídeo Meet My Porto (a partir de 600€)
> 
> *   **Total Estimado:** Este plano representa o auge do luxo e personalização, com serviços adicionais para tornar o seu dia inesquecível. O valor total do plano pode variar bastante também dependendo do número de convidados, mas podemos estimar um valor inicial a partir dos 1300€, sem contar o preço do local por convidado.
> 
> Espero que estas opções iniciais lhe deem uma boa perspetiva dos custos e possibilidades. Qual deles te agrada mais? Ou você gostaria de ajustar algo para adequar melhor ao seu orçamento ou preferências?


In [98]:
def remover_conteudo_ate_substring(texto_completo, marcador_substring):
  """
  Remove o conteúdo de uma string até encontrar um marcador_substring.

  Args:
    texto_completo: A string original.
    marcador_substring: A substring que marca o ponto a partir do qual
                        o conteúdo deve ser mantido.

  Returns:
    A parte da string original após o marcador_substring (incluindo o marcador,
    se quiseres manter a partir dele, ou excluindo-o, como no exemplo).
    Se o marcador não for encontrado, devolve a string original.
  """
  posicao_marcador = texto_completo.find(marcador_substring)

  # Verifica se o marcador foi encontrado
  if posicao_marcador != -1:
    # Calcula a posição onde a substring marcador termina
    fim_marcador = posicao_marcador + len(marcador_substring)
    # Retorna a parte da string após o marcador
    return texto_completo[fim_marcador:]
  else:
    # Se o marcador não for encontrado, retorna a string original
    # print(f"Atenção: O marcador '{marcador_substring}' não foi encontrado na string.")
    return texto_completo


In [99]:
print("🚀 Iniciando o Sistema de Criação de MVP com 4 Agentes 🚀")

# --- Obter o Tópico do Usuário ---
tipo_evento = input("Olá, somos a Kreyzon Events, em que tipo de evento podemos ajudar? ")

# Inserir lógica do sistema de agentes ################################################
if not tipo_evento:
    print("Você esqueceu de digitar o que quer!")
else:
    print(f"Maravilha! Vou falar com a equipe para lhe dar as melhores opções para seu evento.")

    detalhes_evento = agente_cliente(tipo_evento)
    # trace flow
    # print("\n--- 📝 Resultado do cliente ---\n")
    # display(to_markdown(detalhes_evento))
    # print("--------------------------------------------------------------")

    plano_evento = agente_aux_adm(tipo_evento, detalhes_evento)
    # trace flow
    # print("\n--- 📝 Resultado do Product Owner ---\n")
    # display(to_markdown(plano_evento))
    # print("--------------------------------------------------------------")

    # criação
    plano_object = json.loads(remover_conteudo_ate_substring(plano_evento, "```json").replace("```json", "").replace("```", ""))
    budgets = []
    for elemento_essencial in plano_object.get("Elementos Essenciais"):
      budget = agente_analista_budget(tipo_evento, elemento_essencial.get("Categoria"), elemento_essencial.get("Necessidades"))
      budgets.append(budget)

    # trace flow
    # print("\n--- 📝 Resultado do Scrum Master ---\n")
    # display(to_markdown(rascunho_de_post))
    # print("--------------------------------------------------------------")

    evento = agente_organizador(tipo_evento, budgets)
    print("\n--- 📝 Resultado do Evento ---\n")
    display(to_markdown(evento))
    print("--------------------------------------------------------------")
    msg = input("Você gostou de alguma das opções ou tem alguma dúvida sobre o evento? ")
    while msg != 'fim':
        resposta_atendente = agente_atendente(tipo_evento, evento, msg)
        display(to_markdown(resposta_atendente))
        print("--------------------------------------------------------------")
        msg = input()

🚀 Iniciando o Sistema de Criação de MVP com 4 Agentes 🚀
Olá, somos a Kreyzon Events, em que tipo de evento podemos ajudar? Festa de casamento do meu primo que mora em Braga, Portugal
Maravilha! Vou falar com a equipe para lhe dar as melhores opções para seu evento.

--- 📝 Resultado do Evento ---



> Com base nos orçamentos fornecidos, apresento 3 planos resumidos para o casamento do seu primo em Braga:
> 
> **Plano Econômico:**
> 
> *   **Quinta:** Quinta das Carpas (a partir de 82€ por pessoa)
> *   **Catering:** Maria Silva Catering Eventos (a partir de 60€ por pessoa)
> *   **Música:** DJ Sá (desde 150€)
> *   **Decoração:** 30s Festas (a partir de 15€)
> *   **Fotografia:** Fotógrafo Casório (a partir de 490€)
> *   **Alianças:** Alianças Pime Portugal (Aço) (20€)
> *   **Total Estimado:** A partir de 667€ + (142€ por convidado)
> 
> **Plano Intermediário:**
> 
> *   **Quinta:** Quinta da Eira do Sol (a partir de 94€ por pessoa)
> *   **Catering:** Pedrinhas Catering (a partir de 65€ por pessoa)
> *   **Música:** Diogo Sousa DJ (entre 150€ e 850€)
> *   **Decoração:** Flores por Favor Atelier (a partir de 100€)
> *   **Fotografia:** Filmagem Zaask (entre 450€ e 2500€)
> *   **Alianças:** Alianças Pime Portugal (Aço) (20€)
> *   **Total Estimado:** A partir de 819€ + (159€ por convidado)
> 
> **Plano Premium:**
> 
> *   **Quinta:** Paço da Comenda - Premium Events (a partir de 100€ por pessoa)
> *   **Catering:** Penha Eventos (a partir de 70€ por pessoa)
> *   **Música:** Diogo Sousa DJ (entre 150€ e 850€)
> *   **Decoração:** Dom de Fada Eventos (a partir de 1€)
> *   **Fotografia:** Filmagem Zaask (entre 450€ e 2500€)
> *   **Alianças:** Alianças Pime Portugal (Aço) (20€)
> *   **Total Estimado:** A partir de 621€ + (170€ por convidado)
> 
> **Observações:**
> 
> *   Os preços são "a partir de", o que significa que podem variar dependendo das escolhas específicas e do número de convidados.
> *   Recomenda-se entrar em contato com os fornecedores para orçamentos detalhados e personalização dos serviços.
> *   É importante confirmar a disponibilidade das datas com cada fornecedor antes de tomar decisões finais.


--------------------------------------------------------------


KeyboardInterrupt: Interrupted by user

In [93]:
    msg = input("Você gostou de alguma das opções ou tem alguma dúvida sobre o evento? ")
    while msg != 'fim':
        resposta_atendente = agente_atendente(tipo_evento, evento, msg)
        display(to_markdown(resposta_atendente))
        print("--------------------------------------------------------------")
        msg = input()

Você gostou de alguma das opções ou tem alguma dúvida sobre o evento? mostre me os planos


> Olá! Que legal que você está planejando a festa de 4 anos da sua filha! Para começarmos, você já tem alguma ideia de tema para a festa?


--------------------------------------------------------------


KeyboardInterrupt: Interrupted by user